In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 22 21:00:00 2020

@author: Manik Bali
"""

#We have Image data that has images taken from a CCTV in various poses.
#This algorithm identifies individuals even if the images are taken from various poses.
#Takes as input the pose data.
import scipy.io as sio
from   scipy import stats 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from numpy.linalg import eig
import random
import os

#Read in pose data
pose = sio.loadmat(r"C:\Users\Manik Bali\Downloads\pose.mat")['pose']

poses=pose[:,:,:]

image_columns=pose.shape[0]
image_rows=pose.shape[1]
cl=image_columns*image_rows

n_train_images=pose.shape[3]*pose.shape[2] 

cl=image_columns*image_rows

subjects=pose.shape[3]
poses=pose.shape[2]


l_images= np.zeros((subjects*poses,cl),dtype='f')
#l_images= np.zeros((subjects,1920),dtype='f')
mean_class_images =np.zeros((subjects, np.mean(pose[:,:,0:6,0],axis=2).flatten().shape[0]),dtype="f" )
std_class_images  =np.zeros((subjects, np.mean(pose[:,:,0:6,0],axis=2).flatten().shape[0]),dtype="f" )

#for  i in range(n.shape[1]): 
k=0
#Reshape the image arrays into columns. Each column represents a flatten image
for  j in range(subjects):
     for  i in range(poses):
            l_images[k,:]=pose[:,:,i,j].flatten()   
            k=k+1
     mean_class_images[j,:] = np.mean(pose[:,:,:,j],axis=2).flatten() 
     std_class_images[j,:]  = np.std(pose[:,:,:,j],axis=2).flatten() 
                          

     
#combine all the images and make mu=0 and sigma=1
all_images = StandardScaler().fit_transform(l_images) 


training_data= all_images
#Take components as 200
n_components=200

pca=PCA(n_components)
pca.fit(training_data)

cumulative_variance=np.cumsum(pca.explained_variance_ratio_)
plt.plot(cumulative_variance)
         
fig, axes = plt.subplots(3, 8,figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))



proj_data= np.matmul(training_data, pca.components_.T)

#for i, ax in enumerate(axes.flat):
#    print(i,ax)
#    ax.imshow(pca.components_[i].reshape(image_columns, image_rows), cmap='bone')       
#    ax.imshow(proj_data[i].reshape(image_columns, image_rows), cmap='bone')       

#fig, axes = plt.subplots(3, 8,figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))
   
#for i, ax in enumerate(axes.flat):
#    print(i,ax)
#    ax.imshow(training_data[i].reshape(image_columns, image_rows), cmap='bone') 
    
    


fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
#for i in range(10):
#    ax[i, 1].imshow(proj_data[i].reshape(24, 21), cmap='binary_r')
#    ax[i, 0].imshow(all_images[i].reshape(24, 21), cmap='binary_r')


proj_subject= proj_data[:,:]
mean_proj_class =np.zeros((subjects,n_components),dtype="f")
std_proj_class =np.zeros((subjects,n_components),dtype="f")

proj_data_images=np.zeros((poses,subjects,n_components),dtype="f")

#Compute mean of each class ( 68 classes) after  projecting data onto eigen vectors

for  j in range(subjects):
     for i in range(poses):
         proj_data_images[i,j,:]=np.matmul(pose[:,:,i,j].flatten(),pca.components_.T) 
     mean_proj_class[j,:] = np.mean(proj_data_images[:,j,:],axis=0)
     std_proj_class[j,:]  = np.std(proj_data_images[:,j,:],axis=0)     


#Bayes Classifier Pcx=pxc * P(c) 
#Here Pcx is the probability that the image is in a certain class (Nutreal or expression)
#pxc is the liklihood Gaussian MLE P(c) = Prior Probability =  Total nutral or expression images /Total images =100/200=0.5

#x_test= n[:,:,110].flatten() #Testing Neutral

# Input the Class of the test data
 





In [ ]:
j=60
x_test= pose[:,:,1,j].flatten()
 #Project test image onto Eugen Vectors
proj_test_data=np.matmul(x_test, pca.components_.T )

#Construct Liklihood function and compute liklihood
liklihood =np.zeros((n_components,subjects),dtype="float64")
liklihood_class =np.zeros((subjects),dtype="float64")
for j in range(subjects):
    for i in range(n_components):
        mu=mean_proj_class[j,i] 
        sigma=std_proj_class[j,i]          
        liklihood[i,j]=stats.norm.pdf(proj_test_data[i], mu, sigma) 
#        print( i,mu, sigma, proj_test_data[i] ) 
    liklihood_class[j] =np.prod(liklihood[0:50,j])
result = np.where(liklihood_class == np.amax(liklihood_class))
print("Catagory []",result)
print("Liklihood")
#evidence=np.prod(liklihood_expression) + np.prod(liklihood_neutral)